In [1]:
import os

import pandas as pd
from keras.applications import ResNet50
from keras.applications.resnet_v2 import preprocess_input
from numpy import resize
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.python.keras import Input
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.layers import Activation, GlobalAveragePooling2D, Lambda
from tensorflow.python.keras.layers import Conv2D
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.layers import Flatten
from tensorflow.python.keras.layers import MaxPooling2D
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.optimizer_v2.rmsprop import RMSprop
from tensorflow.python.keras.utils.np_utils import to_categorical


2022-08-22 17:37:22.412855: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jacob/Tools/Python/venvs/Default/lib/python3.10/site-packages/cv2/../../lib64:
2022-08-22 17:37:22.412887: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
image_size = (224, 224)
image_size_ext = (224, 224, 3)
batch_size = 5
sample_data = "../../sample-data"


In [3]:
# copilot
def resnet_model(input_shape, num_classes):
    # initialize the model
    model = Sequential()
    # first set of CONV => RELU => POOL layers
    model.add(Conv2D(64, (3, 3), padding="same", input_shape=input_shape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    # second set of CONV => RELU => POOL layers
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    # first (and only) set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    # softmax classifier
    model.add(Dense(num_classes))
    model.add(Activation("softmax"))
    # return the constructed network architecture
    return model



In [4]:
def get_random_images(count: int):
    images = []
    labels = []

    train_data_dir = "../../data/"
    for sub_dir in os.listdir(train_data_dir):
        image_list = os.listdir(
            os.path.join(train_data_dir, sub_dir)
        )  #list of all image names in the directory
        image_list = list(map(lambda x: os.path.join(sub_dir, x), image_list))
        images.extend(image_list)
        labels.extend([sub_dir] * len(image_list))

    df = pd.DataFrame({"Images": images, "Labels": labels})
    df = df.sample(frac=1).reset_index(drop=True)  # To shuffle the data
    df = df.head(count)  # to take the subset of data (I'm taking 100 from it)

    aug = ImageDataGenerator(
        rotation_range=30,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        validation_slip=0.2,
    )

    train_generator = aug.flow_from_dataframe(
        dataframe=df,
        directory=train_data_dir,
        x_col="Images",
        y_col="Labels",
        batch_size=32,
        seed=42,
        shuffle=True,
        class_mode="categorical",
        target_size=image_size,
        subset="training"
    )

    validation_generator = aug.flow_from_dataframe(
        dataframe=df,
        directory=train_data_dir,
        x_col="Images",
        y_col="Labels",
        batch_size=32,
        seed=42,
        shuffle=True,
        class_mode="categorical",
        target_size=image_size,
        subset="validation"
    )
    return train_generator, validation_generator


In [5]:
def read_all_images():
    train_data_gen = ImageDataGenerator(
        rescale=1.0 / 255,
        shear_range=0.5,
        zoom_range=0.7,
        horizontal_flip=True,
        vertical_flip=True
    )

    test_data_gen = ImageDataGenerator(rescale=1.0 / 255)

    train_data = train_data_gen.flow_from_directory(
        "../../sample-data/train/",
        target_size=(224, 224),
        class_mode='categorical'
    )

    test_data = test_data_gen.flow_from_directory(
        "../../sample-data/test/",
        target_size=(224, 224),
        class_mode='categorical'
    )

    return train_data, test_data


In [6]:
read_all_images()


Found 84 images belonging to 4 classes.
Found 0 images belonging to 0 classes.


(<keras.preprocessing.image.DirectoryIterator at 0x7f733c713eb0>,
 <keras.preprocessing.image.DirectoryIterator at 0x7f733c713ca0>)

In [7]:
def read_images_updated():
    train_dataset = image_dataset_from_directory(
        f"{sample_data}/train", shuffle=True, batch_size=batch_size, image_size=image_size
    )

    validation_dataset = image_dataset_from_directory(
        f"{sample_data}/test", shuffle=True, batch_size=batch_size, image_size=image_size
    )


In [8]:
def get_data():
    pass


def preprocess_data(X, Y):
    x_p = preprocess_input(X)
    y_p = to_categorical(Y, 10)
    return x_p, y_p


def preprocess_input_data(train_x, train_y, test_x, test_y):
    train_x, train_y = preprocess_data(train_x, train_y)
    test_x, test_y = preprocess_data(test_x, test_y)
    return (train_x, train_y), (test_x, test_y)


In [9]:
def create_model():
    inputs = Input(shape=image_size_ext)
    resnet = ResNet50(weights="imagenet", include_top=False, input_tensor=inputs)

    for layer in resnet.layers[:170]:
        layer.trainable = False

    model = Sequential()
    model.add(Lambda(lambda x: resize(x, image_size)))
    model.add(resnet)
    model.add(GlobalAveragePooling2D())
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(10, activation='softmax'))

    return model


In [10]:
def fit_model(model, train_x, train_y, test_x, test_y):
    model.compile(
        loss='binary_crossentropy',
        optimizer=RMSprop(lr=2e-5), metrics=['accuracy']
    )
    checkpointer = ModelCheckpoint(
        filepath='cifar10.h5',
        monitor="val_accuracy", verbose=1, save_best_only=True
    )
    model.fit(
        train_x, train_y, batch_size=32, epochs=10, verbose=1,
        callbacks=[checkpointer], validation_data=(test_x, test_y),
        shuffle=True
    )
    model.summary()
    model.save("cifar10.h5")


In [11]:
def training(model):
    train_data_dir = ""
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )  # set validation split

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode="classification",
        subset='training'
    )  # set as training data

    validation_generator = train_datagen.flow_from_directory(
        train_data_dir,  # same directory as training data
        target_size=image_size,
        batch_size=batch_size,
        class_mode="classification",
        subset='validation'
    )  # set as validation data

    model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size,
        epochs=5
    )


ValueError: Unexpectedly found an instance of type `<class 'tensorflow.python.keras.engine.keras_tensor.KerasTensor'>`. Expected a symbolic tensor instance.